In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s6f6cno7
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s6f6cno7
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7432 sha256=ad99cf1e6175f37f13398802e9e82d42885745e385f9d1c2ce678f3d359f3a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-pe9a76cn/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <iostream>
#include <opencv2/opencv.hpp>
#include <cuda_runtime.h>

// CUDA kernel for computing average color
__global__ void computeAverageColor(const uchar* image, int width, int height, float* result) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int idx = y * width + x;
        atomicAdd(&result[0], static_cast<float>(image[3 * idx]));       // Red channel
        atomicAdd(&result[1], static_cast<float>(image[3 * idx + 1]));   // Green channel
        atomicAdd(&result[2], static_cast<float>(image[3 * idx + 2]));   // Blue channel
    }
}

int main() {
    // Load image using OpenCV
    cv::Mat image = cv::imread("image.jpg");

    if (image.empty()) {
        std::cerr << "Error loading image." << std::endl;
        return -1;
    }

    // Allocate memory for CUDA
    uchar* d_image;
    float* d_result;
    cudaMalloc((void**)&d_image, image.rows * image.cols * 3 * sizeof(uchar));
    cudaMalloc((void**)&d_result, 3 * sizeof(float));

    // Copy image data to device
    cudaMemcpy(d_image, image.data, image.rows * image.cols * 3 * sizeof(uchar), cudaMemcpyHostToDevice);

    // Set up CUDA grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((image.cols + blockDim.x - 1) / blockDim.x, (image.rows + blockDim.y - 1) / blockDim.y);

    // Launch CUDA kernel
    computeAverageColor<<<gridDim, blockDim>>>(d_image, image.cols, image.rows, d_result);

    // Copy result back to host
    float h_result[3];
    cudaMemcpy(h_result, d_result, 3 * sizeof(float), cudaMemcpyDeviceToHost);

    // Calculate average color
    h_result[0] /= (image.rows * image.cols);
    h_result[1] /= (image.rows * image.cols);
    h_result[2] /= (image.rows * image.cols);

    // Display result
    std::cout << "Average Color: (" << h_result[0] << ", " << h_result[1] << ", " << h_result[2] << ")" << std::endl;

    // Free CUDA memory
    cudaFree(d_image);
    cudaFree(d_result);

    return 0;
}


In [ ]:



%%cu
#include <iostream>
#include <opencv2/opencv.hpp>
#include <cuda_runtime.h>

// CUDA kernel for shape feature extraction
__global__ void extractShapeFeatures(const uchar* image, int width, int height, int* result) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int idx = y * width + x;

        // Use Canny edge detector to find edges
        uchar edge = cv::saturate_cast<uchar>(cv::Canny(image[idx], 50, 150));

        // Count pixels belonging to edges in each block
        atomicAdd(&result[blockIdx.x + blockIdx.y * gridDim.x], edge);
    }
}

int main() {
    // Load image using OpenCV
    cv::Mat image = cv::imread("image.jpg", cv::IMREAD_GRAYSCALE);

    if (image.empty()) {
        std::cerr << "Error loading image." << std::endl;
        return -1;
    }

    // Allocate memory for CUDA
    uchar* d_image;
    int* d_result;
    cudaMalloc((void**)&d_image, image.rows * image.cols * sizeof(uchar));
    cudaMalloc((void**)&d_result, image.cols * image.rows * sizeof(int));

    // Copy image data to device
    cudaMemcpy(d_image, image.data, image.rows * image.cols * sizeof(uchar), cudaMemcpyHostToDevice);

    // Set up CUDA grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((image.cols + blockDim.x - 1) / blockDim.x, (image.rows + blockDim.y - 1) / blockDim.y);

    // Launch CUDA kernel
    extractShapeFeatures<<<gridDim, blockDim>>>(d_image, image.cols, image.rows, d_result);

    // Copy result back to host
    int* h_result = new int[gridDim.x * gridDim.y];
    cudaMemcpy(h_result, d_result, gridDim.x * gridDim.y * sizeof(int), cudaMemcpyDeviceToHost);

    // Calculate total number of edge pixels
    int totalEdgePixels = 0;
    for (int i = 0; i < gridDim.x * gridDim.y; ++i) {
        totalEdgePixels += h_result[i];
    }

    // Display result
    std::cout << "Total Edge Pixels: " << totalEdgePixels << std::endl;

    // Free CUDA memory and host memory
    cudaFree(d_image);
    cudaFree(d_result);
    delete[] h_result;

    return 0;
}


In [ ]:
sudo apt-get install libopencv-cuda-dev


In [ ]:
#include <iostream>
#include <opencv2/opencv.hpp>
#include <opencv2/cudaimgproc.hpp>
#include <opencv2/cudaarithm.hpp>

// CUDA kernel for size feature extraction
__global__ void extractSizeFeatures(const uchar* image, int width, int height, int* result) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int idx = y * width + x;

        // Assume pixels above a certain intensity threshold as object pixels
        uchar objectPixel = (image[idx] > 128) ? 1 : 0;

        // Count object pixels in each block
        atomicAdd(&result[blockIdx.x + blockIdx.y * gridDim.x], objectPixel);
    }
}

int main() {
    // Load image using OpenCV
    cv::Mat image = cv::imread("image.jpg", cv::IMREAD_GRAYSCALE);

    if (image.empty()) {
        std::cerr << "Error loading image." << std::endl;
        return -1;
    }

    // Allocate memory for CUDA
    uchar* d_image;
    int* d_result;
    cudaMalloc((void**)&d_image, image.rows * image.cols * sizeof(uchar));
    cudaMalloc((void**)&d_result, image.cols * image.rows * sizeof(int));

    // Copy image data to device
    cudaMemcpy(d_image, image.data, image.rows * image.cols * sizeof(uchar), cudaMemcpyHostToDevice);

    // Set up CUDA grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((image.cols + blockDim.x - 1) / blockDim.x, (image.rows + blockDim.y - 1) / blockDim.y);

    // Launch CUDA kernel
    extractSizeFeatures<<<gridDim, blockDim>>>(d_image, image.cols, image.rows, d_result);

    // Copy result back to host
    int* h_result = new int[gridDim.x * gridDim.y];
    cudaMemcpy(h_result, d_result, gridDim.x * gridDim.y * sizeof(int), cudaMemcpyDeviceToHost);

    // Calculate total number of object pixels
    int totalObjectPixels = 0;
    for (int i = 0; i < gridDim.x * gridDim.y; ++i) {
        totalObjectPixels += h_result[i];
    }

    // Display result
    std::cout << "Total Object Pixels: " << totalObjectPixels << std::endl;

    // Free CUDA memory and host memory
    cudaFree(d_image);
    cudaFree(d_result);
    delete[] h_result;

    return 0;
}


In [ ]:
#include <iostream>
#include <opencv2/opencv.hpp>
#include <opencv2/cudaimgproc.hpp>
#include <opencv2/cudaarithm.hpp>

// CUDA kernel for LBP feature extraction
__global__ void extractTextureFeatures(const uchar* image, int width, int height, uchar* result) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x > 0 && y > 0 && x < width - 1 && y < height - 1) {
        int idx = y * width + x;

        // Calculate LBP value
        uchar lbp = 0;
        uchar center = image[idx];

        lbp |= (image[(y - 1) * width + x - 1] > center) << 7;
        lbp |= (image[(y - 1) * width + x] > center) << 6;
        lbp |= (image[(y - 1) * width + x + 1] > center) << 5;
        lbp |= (image[y * width + x + 1] > center) << 4;
        lbp |= (image[(y + 1) * width + x + 1] > center) << 3;
        lbp |= (image[(y + 1) * width + x] > center) << 2;
        lbp |= (image[(y + 1) * width + x - 1] > center) << 1;
        lbp |= (image[y * width + x - 1] > center);

        result[idx] = lbp;
    }
}

int main() {
    // Load image using OpenCV
    cv::Mat image = cv::imread("image.jpg", cv::IMREAD_GRAYSCALE);

    if (image.empty()) {
        std::cerr << "Error loading image." << std::endl;
        return -1;
    }

    // Allocate memory for CUDA
    uchar* d_image;
    uchar* d_result;
    cudaMalloc((void**)&d_image, image.rows * image.cols * sizeof(uchar));
    cudaMalloc((void**)&d_result, image.rows * image.cols * sizeof(uchar));

    // Copy image data to device
    cudaMemcpy(d_image, image.data, image.rows * image.cols * sizeof(uchar), cudaMemcpyHostToDevice);

    // Set up CUDA grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((image.cols + blockDim.x - 1) / blockDim.x, (image.rows + blockDim.y - 1) / blockDim.y);

    // Launch CUDA kernel
    extractTextureFeatures<<<gridDim, blockDim>>>(d_image, image.cols, image.rows, d_result);

    // Copy result back to host
    uchar* h_result = new uchar[image.rows * image.cols];
    cudaMemcpy(h_result, d_result, image.rows * image.cols * sizeof(uchar), cudaMemcpyDeviceToHost);

    // Display result
    std::cout << "Texture Features: ";
    for (int i = 0; i < image.rows * image.cols; ++i) {
        std::cout << static_cast<int>(h_result[i]) << " ";
    }
    std::cout << std::endl;

    // Free CUDA memory and host memory
    cudaFree(d_image);
    cudaFree(d_result);
    delete[] h_result;

    return 0;
}
